In [1]:
from datascience import *
import numpy as np, pandas as pd, matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)

In [2]:
pandas_table = pd.read_json("Sarcasm_Headlines_Dataset_v2.json", lines= True)
articles = Table.from_df(pandas_table)
articles = articles.with_column('length', articles.apply(len, 'headline'))
articles

article_link,headline,is_sarcastic,length
https://www.theonion.com/thirtysomething-scientists-unve ...,thirtysomething scientists unveil doomsday clock of hair ...,1,61
https://www.huffingtonpost.com/entry/donna-edwards-inequ ...,dem rep. totally nails why congress is falling short on ...,0,79
https://www.huffingtonpost.com/entry/eat-your-veggies-9- ...,eat your veggies: 9 deliciously different recipes,0,49
https://local.theonion.com/inclement-weather-prevents-li ...,inclement weather prevents liar from getting to work,1,52
https://www.theonion.com/mother-comes-pretty-close-to-us ...,mother comes pretty close to using word 'streaming' corr ...,1,61
https://www.huffingtonpost.com/entry/my-white-inheritanc ...,my white inheritance,0,20
https://www.huffingtonpost.com/entry/5-ways-to-file-your ...,5 ways to file your taxes with less stress,0,42
https://www.theonion.com/richard-bransons-global-warming ...,richard branson's global-warming donation nearly as much ...,1,88
https://politics.theonion.com/shadow-government-getting- ...,shadow government getting too large to meet in marriott ...,1,73
https://www.huffingtonpost.comhttp://pubx.co/6IXxhm,lots of parents know this scenario,0,34


In [3]:
sarcastic = articles.where('is_sarcastic', 1)
real = articles.where('is_sarcastic', 0)

In [4]:
sarcastic_headlines = sarcastic.select('headline')
real_headlines = real.select('headline')

In [22]:
useless_words = ['to', 'of', 'in', 'for', 'on', 'by', 'the', 'it', 'a', 'at']

In [28]:
sarcastic_words = []

for i in sarcastic_headlines.column(0):
    for j in i.split():
        sarcastic_words.append(j)

sarcastic_tbl = Table().with_column('words', sarcastic_words)
sarcastic_tbl = sarcastic_tbl.group('words').where('words', are.not_contained_in(useless_words)).sort(1, descending=True)
sarcastic_tbl

words,count
man,1143
with,1008
new,973
at,673
from,664
out,559
after,546
up,507
he,499
area,487
